In [1]:
import mysql.connector
import openai
from dotenv import load_dotenv
import os
import logging
import time
from functools import wraps
from pathlib import Path

In [2]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Specify the path to .env file in the project root
env_path = Path('../.env')
load_dotenv(dotenv_path=env_path)


# Check if the environment variables are loaded
db_name = os.getenv('DB_NAME')
logging.info(f"Loaded database name: {db_name}")

if db_name is None:
    logging.error("Error: Could not load .env file or DB_NAME is not defined")
    raise EnvironmentError("DB_NAME is not defined in .env file")

2024-10-27 16:07:32,819 - INFO - Loaded database name: CustomerSales


In [3]:
# Decorador para medir el tiempo de ejecución de las funciones
def time_execution(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        logging.info(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
        return result
    return wrapper

# Decorador para capturar excepciones y registrarlas
def log_exceptions(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Error occurred in function '{func.__name__}': {e}")
            raise e
    return wrapper

In [4]:
# Clase para gestionar la conexión a MySQL
class MySQLDatabase:
    def __init__(self):
        self.host = os.getenv('DB_HOST')
        self.user = os.getenv('DB_USER')
        self.password = os.getenv('DB_PASSWORD')
        self.database = os.getenv('DB_NAME')
        self.port = os.getenv('DB_PORT')
        self.connection = None

    @log_exceptions
    @time_execution
    def connect(self):
        if self.connection is not None and self.connection.is_connected():
            logging.info("Already connected to the database.")
            return
        logging.info("Connecting to MySQL database...")
        self.connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database,
            port=self.port
        )
        logging.info("Successfully connected to the database.")

    @log_exceptions
    @time_execution
    def execute_query(self, query, params=None):
        if self.connection is None or not self.connection.is_connected():
            self.connect()
        logging.info(f"Executing query: {query}")
        cursor = self.connection.cursor(dictionary=True)  # Using dictionary cursor for better handling
        cursor.execute(query, params)
        result = cursor.fetchall()
        logging.info("Query executed successfully.")
        cursor.close()
        return result

    @log_exceptions
    @time_execution
    def close(self):
        if self.connection and self.connection.is_connected():
            self.connection.close()
            logging.info("Connection closed.")

In [5]:
# Crear instancia de la base de datos
db = MySQLDatabase()
# Conectar a la base de datos
db.connect()

2024-10-27 16:07:36,687 - INFO - Connecting to MySQL database...
2024-10-27 16:07:36,715 - INFO - Successfully connected to the database.
2024-10-27 16:07:36,715 - INFO - Function 'connect' executed in 0.0279 seconds


In [6]:
# ejecutar una consulta SQL
query = "SELECT * FROM ClientesVentas LIMIT 5;"
result = db.execute_query(query)
logging.info(f"Query Result: {result}")

2024-10-27 16:07:37,844 - INFO - Executing query: SELECT * FROM ClientesVentas LIMIT 5;
2024-10-27 16:07:37,848 - INFO - Query executed successfully.
2024-10-27 16:07:37,848 - INFO - Function 'execute_query' executed in 0.0041 seconds
2024-10-27 16:07:37,849 - INFO - Query Result: [{'id': 1, 'nombre': 'Juan', 'apellidos': 'Pérez', 'edad': 35, 'marca_producto': 'Apple', 'importe_vendido': Decimal('1200.50')}, {'id': 2, 'nombre': 'Ana', 'apellidos': 'García', 'edad': 28, 'marca_producto': 'Samsung', 'importe_vendido': Decimal('950.75')}, {'id': 3, 'nombre': 'Luis', 'apellidos': 'Martínez', 'edad': 42, 'marca_producto': 'Sony', 'importe_vendido': Decimal('560.00')}, {'id': 4, 'nombre': 'Carmen', 'apellidos': 'Fernández', 'edad': 30, 'marca_producto': 'LG', 'importe_vendido': Decimal('1340.10')}, {'id': 5, 'nombre': 'Pedro', 'apellidos': 'López', 'edad': 45, 'marca_producto': 'Apple', 'importe_vendido': Decimal('2500.00')}]


In [13]:
from openai import OpenAI
import os

class NaturalLanguageToSQL:
    def __init__(self):
        self.api_key = os.getenv('OPENAI_API_KEY')
        openai.api_key = self.api_key

    @log_exceptions
    def translate(self, natural_language_query):
        messages = [
            {"role": "system", "content": "Eres un asistente que traduce consultas en lenguaje natural a SQL usando la tabla ClientesVentas, que contiene las columnas: id, nombre, apellidos, edad, marca_producto e importe_vendido."},
            {"role": "user", "content": natural_language_query}
        ]
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=100,
            temperature=0
        )
        sql_query = response['choices'][0]['message']['content'].strip()
    
        # Limpiar la consulta eliminando caracteres innecesarios
        if sql_query.startswith("```sql"):
            sql_query = sql_query[5:]
        if sql_query.endswith("```"):
            sql_query = sql_query[:-3]
    
        # Eliminar caracteres adicionales que puedan estar fuera de lugar
        sql_query = sql_query.lstrip("l").strip()  # Eliminar el 'l' al inicio de la consulta si aparece
        
        return sql_query.strip()

In [14]:
# Clase principal para gestionar el flujo completo
class QueryProcessor:
    def __init__(self):
        self.db = MySQLDatabase()
        self.nl_to_sql = NaturalLanguageToSQL()

    @log_exceptions
    @time_execution
    def process_query(self, natural_language_query):
        # 1. Convertir el lenguaje natural a SQL
        sql_query = self.nl_to_sql.translate(natural_language_query)
        logging.info(f"SQL Query: {sql_query}")

        # 2. Ejecutar la consulta SQL
        result = self.db.execute_query(sql_query)

        # 3. Devolver el resultado y la consulta SQL generada
        return {"sql_query": sql_query, "result": result}

    @log_exceptions
    @time_execution
    def close(self):
        # Cerrar la conexión a la base de datos
        self.db.close()

In [15]:
# Crear instancia del procesador de consultas
query_processor = QueryProcessor()

In [16]:
# Your natural language query
natural_language_query = "Show me average importe_vendido of customers"

# Procesar la consulta
result = query_processor.process_query(natural_language_query)
logging.info(f"SQL Query: {result['sql_query']}")
logging.info(f"Query Result: {result['result']}")

2024-10-27 16:08:19,666 - ERROR - Error occurred in function 'translate': 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

2024-10-27 16:08:19,666 - ERROR - Error occurred in function 'process_query': 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migrat

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [17]:
# Your natural language query
natural_language_query = "Show me the names and surnames of customers older than 30 years and who sold more than 1000 euros."

# Procesar la consulta
result = query_processor.process_query(natural_language_query)
logging.info(f"SQL Query: {result['sql_query']}")
logging.info(f"Query Result: {result['result']}")

2024-10-27 16:08:20,218 - ERROR - Error occurred in function 'translate': 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

2024-10-27 16:08:20,223 - ERROR - Error occurred in function 'process_query': 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migrat

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [18]:
# Cerrar la conexión
query_processor.close()

2024-10-27 16:08:20,890 - INFO - Function 'close' executed in 0.0000 seconds
2024-10-27 16:08:20,891 - INFO - Function 'close' executed in 0.0013 seconds
